# Plate Velocity Basics

This notebook goes through a series of examples for how to use pygplates to work with plate velocities.

The basic requirements to calculate any plate velocities are:
  - a rotation file containing Euler poles defining plate motions
  - the plateid(s) that define the moving plate (and the frame of reference)
  - a time range over which to determine the velocity
  - the geographic location of the point where we evaluate velocity
  
### Example #1

The first example illustrates these concepts:
  - First, we load the rotation model from the Seton et al (2012) dataset
  - Second, we get the stage rotation that describes how the plate 101 (North America) is moving over the time period between 11 Ma and 10 Ma. [Note that since don't specify which plate North America is moving relative to, this calculation will default to getting the velcocity relative to the absolute reference frame in the rotation file]


In [ ]:
import pygplates
import numpy as np

rotation_filename = '../Data/Seton_etal_ESR2012_2012.1.rot'

rotation_model = pygplates.RotationModel(rotation_filename)

timeFrom = 11.
timeTo = 10.
MovingPlate = 101

# Get the rotation from 11Ma to 10Ma, and the feature's reconstruction plate ID.
equivalent_stage_rotation = rotation_model.get_rotation(
    timeTo, MovingPlate, timeFrom)


- Having determined the stage rotation, define a specific point where we are going to evaluate the velocity 

In [11]:
velocity_point = pygplates.PointOnSphere((20,-60))

# Calculate a velocity for each reconstructed point over the 1My time interval.
velocity_vector = pygplates.calculate_velocities(
    velocity_point,
    equivalent_stage_rotation,
    1,
    pygplates.VelocityUnits.cms_per_yr)

velocity_magnitude_azimuth = pygplates.LocalCartesian.convert_from_geocentric_to_magnitude_azimuth_inclination(
                        velocity_point,
                        velocity_vector)

print velocity_vector
print velocity_magnitude_azimuth


[(10.129780064740592, 5.87816230240699, 4.383996562727298e-17)]


Each entry in the list 'velocity_magnitude_azimuth' contains 3 values, where the first value is the magnitude, and the second is the azimuth:

In [12]:
print 'Velocity: magnitude = %0.4f cm/yr, azimuth = %0.4f' % \
            (velocity_magnitude_azimuth[0][0],np.degrees(velocity_magnitude_azimuth[0][1]))

Velocity: magnitude = 10.1298 cm/yr, azimuth = 336.7939


### Example #2

The next cell is follows the same approach, but evaluates the velocity at a series of times. In this case, the point is assigned a plateid of 801 (Australia) and this point is then reconstructed relative to 802 (Antarctica), so that we can get the velocity of the same point on Australia

Note also that this second example also defines a 'delta' time, which controls the length of time that the cmputed velocity is averaged over (ie here the velocity is computed based on the motion for 0-10 Ma, 10-20 Ma, etc,  because the delta time is 10 Myr, you can choose this value). 

In [14]:
# Create for each point we want to reconstruct 
points = []
points.append((-30.,110.,801))

point_features = []
for lat, lon, plate_id in points:
    point_feature = pygplates.Feature()
    point_feature.set_geometry(pygplates.PointOnSphere(lat, lon))
    point_feature.set_reconstruction_plate_id(plate_id)
    point_features.append(point_feature)

max_time = 100.
delta_time = 10.

fixed_plate = 802

for time in np.arange(0,max_time+1.,delta_time):    
    # Reconstruct the point features.
    reconstructed_feature_geometries = []
    pygplates.reconstruct(point_features, rotation_model, reconstructed_feature_geometries, time)
    
    # Get the rotation from 'time+delta' to 'time', and the feature's reconstruction plate ID.
    equivalent_stage_rotation = rotation_model.get_rotation(
        time, plate_id, time+delta_time)
    
    for reconstructed_feature_geometry in reconstructed_feature_geometries:      
        # Calculate a velocity for each reconstructed point over the delta time interval.
        velocity_vector = pygplates.calculate_velocities(
            reconstructed_feature_geometry.get_reconstructed_geometry(),
            equivalent_stage_rotation,
            delta_time,
            pygplates.VelocityUnits.cms_per_yr)

        velocity_magnitude_azimuth = pygplates.LocalCartesian.convert_from_geocentric_to_magnitude_azimuth_inclination(
                                reconstructed_feature_geometry.get_reconstructed_geometry(),
                                velocity_vector)
    
        print 'Time = %0.2f Ma' % time
        print 'Reconstructed Point Lat, Long = %s, %s' % \
            reconstructed_feature_geometry.get_reconstructed_geometry().to_lat_lon()
        print 'Velocity: magnitude = %0.4f cm/yr, azimuth = %0.4f' % \
            (velocity_magnitude_azimuth[0][0],np.degrees(velocity_magnitude_azimuth[0][1]))


Time = 0.00 Ma
Reconstructed Point Lat, Long = -30.0, 110.0
Velocity: magnitude = 6.6331 cm/yr, azimuth = 19.4978
Time = 10.00 Ma
Reconstructed Point Lat, Long = -34.7034363088, 106.320362839
Velocity: magnitude = 6.5950 cm/yr, azimuth = 22.1596
Time = 20.00 Ma
Reconstructed Point Lat, Long = -39.7199742518, 102.619170784
Velocity: magnitude = 7.1452 cm/yr, azimuth = 22.6106
Time = 30.00 Ma
Reconstructed Point Lat, Long = -45.4420121063, 98.1338664523
Velocity: magnitude = 5.9355 cm/yr, azimuth = 30.5566
Time = 40.00 Ma
Reconstructed Point Lat, Long = -49.7851226802, 92.5918766476
Velocity: magnitude = 1.7138 cm/yr, azimuth = 7.3122
Time = 50.00 Ma
Reconstructed Point Lat, Long = -50.502462132, 92.3950824325
Velocity: magnitude = 0.8073 cm/yr, azimuth = 304.3038
Time = 60.00 Ma
Reconstructed Point Lat, Long = -55.2654778195, 94.7532443839
Velocity: magnitude = 0.7851 cm/yr, azimuth = 300.9687
Time = 70.00 Ma
Reconstructed Point Lat, Long = -55.5242783688, 93.4581634027
Velocity: magnit